In [1]:
import pandas as pd
import numpy as np
import glob
import os
import re
from datetime import datetime, timedelta

### Load data

In [2]:
def load_and_concat_csv(folder_path, chunksize=None):
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    df_list = []

    for filename in all_files:
        try:
            # Read the CSV file, potentially in chunks
            df_chunks = pd.read_csv(filename, chunksize=chunksize, 
                                    low_memory=False, encoding='utf-8')
            
            if chunksize:
                # If reading in chunks, concatenate the chunks
                df = pd.concat(df_chunks, ignore_index=True)
            else:
                df = next(df_chunks)
            
            df['source_file'] = os.path.basename(filename)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading file {filename}: {str(e)}")

    combined_df = pd.concat(df_list, ignore_index=True, sort=False)
    
    # Handle missing values
    #combined_df = combined_df.fillna('Unknown')  # or use another appropriate method
    
    return combined_df

In [3]:
folder_path = "../output/"
result_df = load_and_concat_csv(folder_path, chunksize=100000)

print(result_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8029471 entries, 0 to 8029470
Data columns (total 23 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   Rank                               object 
 1   Performance                        object 
 2   Surname, first name                object 
 3   Club                               object 
 4   Nat.                               object 
 5   YOB                                object 
 6   M/F                                object 
 7   Rank M/F                           float64
 8   Cat                                object 
 9   Cat. Rank                          float64
 10  Avg.Speed km/h                     object 
 11  Age graded performance             object 
 12  Runner ID                          object 
 13  Event                              object 
 14  Date                               object 
 15  Distance                           object 
 16  Finishers         

In [ ]:
def sample_csv(input_file, output_file, n, random_state=None):
    """
    Sample n rows from a CSV file and save to a new CSV file.
    
    :param input_file: Path to the input CSV file
    :param output_file: Path to save the output CSV file
    :param n: Number of rows to sample
    :param random_state: Seed for random number generator (optional)
    """
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Sample n rows
    sampled_df = df.sample(n=n, random_state=random_state)
    
    # Save the sampled DataFrame to a new CSV file
    sampled_df.to_csv(output_file, index=False)
    
    print(f"Sampled {n} rows from {input_file} and saved to {output_file}")

# Usage
input_file = '../output/all_events_data_2024.csv'
output_file = '2024_sample.csv'
sample_size = 500 

sample_csv(input_file, output_file, sample_size)

### Clean data/Feature eng

In [60]:
def split_runner_name(data):
    # Initialize 'Surname' and 'First Name' columns with empty strings
    data['Surname'] = ''
    data['First Name'] = ''
    
    # Check 'Surname, first name' column first
    if 'Surname, first name' in data.columns:
        mask = data['Surname, first name'].notna()
        temp = data.loc[mask, 'Surname, first name'].str.split(',', expand=True)
        data.loc[mask, 'Surname'] = temp[0].str.strip()
        data.loc[mask, 'First Name'] = temp[1].str.strip() if temp.shape[1] > 1 else ''
    
    # Check 'Original name\nSurname, first name' column if 'Surname' is still empty
    original_name_column = 'Original name\nSurname, first name'
    if original_name_column in data.columns:
        mask = (data['Surname'] == '') & (data[original_name_column].notna())
        temp = data.loc[mask, original_name_column].str.split(',', expand=True)
        data.loc[mask, 'Surname'] = temp[0].str.strip()
        data.loc[mask, 'First Name'] = temp[1].str.strip() if temp.shape[1] > 1 else ''
    
    return data

def parse_performance(data, column_name, time_column, distance_column):
    time_regex = re.compile(r'(?:(\d+)d )?(\d{1,2}):(\d{2}):(\d{2}) h')
    distance_regex = re.compile(r'(\d+\.?\d*) km')
    
    def parse_entry(entry):
        if 'km' in entry:
            match = distance_regex.search(entry)
            return (None, float(match.group(1)) if match else None)
        else:
            match = time_regex.search(entry)
            if match:
                days, hours, minutes, seconds = match.groups(default='0')
                total_seconds = timedelta(days=int(days), hours=int(hours), minutes=int(minutes), seconds=int(seconds)).total_seconds()
                return (total_seconds, None)
        return (None, None)
    
    data[time_column], data[distance_column] = zip(*data[column_name].apply(parse_entry))
    return data


def split_distance_column(df):
    # Updated patterns to handle more unit variations
    distance_pattern = re.compile(r'(\d+(?:\.\d*)?)\s*\.?\s*(km|k|mi|m|mile|miles|h|d)', re.IGNORECASE)
    race_type_pattern = re.compile(r'(\d+(?:\.\d*)?)\s*\.?\s*(km|k|mi|m|mile|miles|h|d)\s*(.*)', re.IGNORECASE)
    
    def standardize_unit(unit):
        unit = unit.lower()
        if unit in ['k', 'km']:
            return 'km'
        elif unit in ['m', 'mi', 'mile', 'miles']:
            return 'mi'
        else:
            return unit  # 'h' and 'd' remain unchanged
    
    def split_distance(entry):
        if pd.isna(entry):
            return None, None, None
        
        # Convert to string if it's not already
        entry = str(entry)
        
        distance_match = distance_pattern.search(entry)
        race_type_match = race_type_pattern.search(entry)
        
        if distance_match:
            # Clean up the distance format
            distance_value = distance_match.group(1).rstrip('.')
            unit = standardize_unit(distance_match.group(2))
            distance = f"{distance_value}{unit}"
            
            # Update race_type logic
            if unit == 'h':
                race_type = 'Time'
            elif unit == 'd':
                race_type = 'Multi-day'
            else:
                race_type = 'Distance'
        else:
            distance = None
            race_type = None
        
        # Extract terrain
        if race_type_match:
            terrain_start = race_type_match.end(2)
            terrain = entry[terrain_start:].strip()
        else:
            terrain = None
        
        return distance, terrain, race_type
    
    df[['Distance/Time', 'Terrain', 'Event Type']] = df['Distance'].apply(split_distance).apply(pd.Series)
    return df

def convert_miles_to_km(entry):
    if pd.isna(entry):
        return None
    match = re.match(r'(\d+\.?\d*)(mi|km)', str(entry))
    if match:
        distance, unit = float(match.group(1)), match.group(2)
        return distance * 1.6 if unit == 'mi' else distance
    return None

def standardize_terrain(terrain):
    terrain = str(terrain).lower()
    if 'trail' in terrain:
        return 'trail'
    elif 'road' in terrain:
        return 'road'
    elif 'track' in terrain:
        return 'track'
    else:
        return 'other'


def parse_date_range(date_str):
    if pd.isna(date_str):
        return None
    
    date_str = str(date_str).strip()
    
    # If it's already in the correct format, return as is
    if re.match(r'\d{2}\.\d{2}\.\d{4}', date_str):
        return date_str
    
    # If it's a single date without year, add the current year
    if re.match(r'\d{1,2}\.\d{1,2}\.?$', date_str):
        return f"{date_str.rstrip('.')}1970"  # Using 1970 as a placeholder year
    
    # If it's a date range
    if '-' in date_str:
        start, end = date_str.split('-')
        start = start.strip()
        end = end.strip()
        
        # Extract day, month, and year components
        start_parts = re.findall(r'\d+', start)
        end_parts = re.findall(r'\d+', end)
        
        # Ensure we have at least day and month for start date
        if len(start_parts) < 2:
            return None  # Return None if format is unexpected
        
        # Get year from end date if available, otherwise use current year
        year = end_parts[-1] if len(end_parts) == 3 else str(datetime.now().year)
        
        # Construct the full start date
        return f"{start_parts[0].zfill(2)}.{start_parts[1].zfill(2)}.{year}"
    
    # If it's a single date with year
    if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_str):
        parts = date_str.split('.')
        return f"{parts[0].zfill(2)}.{parts[1].zfill(2)}.{parts[2]}"
    
    # If we can't parse the date, return None
    return None


def extract_location(event):
    match = re.search(r'\((\w+)\)$', event)
    return match.group(1) if match else 'Unknown'

def categorize_age_group(age):
    if pd.isna(age):
        return 'Unknown'
    elif age < 20:
        return 'Under 20'
    elif 20 <= age < 30:
        return '20-29'
    elif 30 <= age < 40:
        return '30-39'
    elif 40 <= age < 50:
        return '40-49'
    elif 50 <= age < 60:
        return '50-59'
    elif 60 <= age < 70:
        return '60-69'
    else:
        return '70+'

def add_age_group(df):
    df['Age Group'] = df['Age'].apply(categorize_age_group)
    return df

def add_runner_statistics(df):
    # Sort the dataframe by Runner ID and Date
    df = df.sort_values(['Runner ID', 'Date'])
    
    # Group by Runner ID
    grouped = df.groupby('Runner ID')
    
    # Number of races (Experience Level) - subtract 1 to exclude current race
    df['Race Count'] = grouped.cumcount()
    
    # Determine the distance to use for cumulative calculation
    df['Distance For Cumulative'] = df.apply(
        lambda row: row['Distance Finish'] if row['Event Type'] == 'Time' else row['Distance KM'],
        axis=1
    )
    
    # Cumulative sum of Distance (excluding current race)
    df['Cumulative Distance KM'] = grouped['Distance For Cumulative'].transform(
        lambda x: x.shift().cumsum()
    )
    
    # Rolling average of Winner Percentage (excluding current race)
    df['Avg Winner Percentage'] = grouped['Winner Percentage'].transform(
        lambda x: x.shift().expanding().mean()
    )
    
    # Remove the temporary column
    df = df.drop('Distance For Cumulative', axis=1)
    
    # Replace NaN values with 0 for first race of each runner
    df['Race Count'] = df['Race Count'].fillna(0)
    df['Cumulative Distance KM'] = df['Cumulative Distance KM'].fillna(0)
    df['Avg Winner Percentage'] = df['Avg Winner Percentage'].fillna(0)
    
    return df

def extract_finishers(df):
    def parse_finishers(finishers_str):
        match = re.match(r'(\d+)\s*\((\d+)\s*M,\s*(\d+)\s*F\)', finishers_str)
        if match:
            total = int(match.group(1))
            male = int(match.group(2))
            female = int(match.group(3))
            return total, male, female
        else:
            return None, None, None

    df[['Total Finishers', 'Male Finishers', 'Female Finishers']] = df['Finishers'].apply(parse_finishers).apply(pd.Series)
    return df


def add_elevation_gain_per_km(df):
    # Calculate elevation gain per km
    df['Elevation Gain per KM'] = df.apply(
        lambda row: row['Elevation Gain'] / row['Distance KM'] 
        if pd.notnull(row['Elevation Gain']) and pd.notnull(row['Distance KM']) and row['Distance KM'] != 0 
        else np.nan, 
        axis=1
    )
    
    # Calculate the median elevation gain per km for each terrain type
    median_elevation_by_terrain = df.groupby('Terrain')['Elevation Gain per KM'].median()
    
    # Fill NaN values with the median for the corresponding terrain type
    df['Elevation Gain per KM'] = df.apply(
        lambda row: median_elevation_by_terrain[row['Terrain']] 
        if pd.isnull(row['Elevation Gain per KM']) and row['Terrain'] in median_elevation_by_terrain
        else row['Elevation Gain per KM'],
        axis=1
    )
    
    return df

def calculate_winner_percentage(df):
    # For distance races (where Event Type is 'Distance')
    distance_mask = df['Event Type'] == 'Distance'
    df.loc[distance_mask, 'Winner Percentage'] = (1 - df.loc[distance_mask, 'Time Seconds Winner'] / df.loc[distance_mask, 'Time Seconds Finish']).round(2)

    # For time races (where Event Type is 'Time')
    time_mask = df['Event Type'] == 'Time'
    df.loc[time_mask, 'Winner Percentage'] = (1 - df.loc[time_mask, 'Distance Finish'] / df.loc[time_mask, 'Distance Winner']).round(2)

    return df


In [25]:
def clean_data(df):
    df = split_runner_name(df)
    df = split_distance_column(df)
    df = parse_performance(df, 'Performance', 'Time Seconds Finish', 'Distance Finish')
    df = parse_performance(df, 'Winner Time', 'Time Seconds Winner', 'Distance Winner')
    df = calculate_winner_percentage(df)
    df = extract_finishers(df)
    
    df['Terrain'] = df['Terrain'].apply(standardize_terrain)
    df['Distance KM'] = df['Distance/Time'].apply(convert_miles_to_km).round(0)
    df['Finish Percentage'] = (df['Rank'] / df['Total Finishers']).round(2)
    df['Distance KM'] = df['Distance KM'].replace(0, pd.NA)
    df['Average Speed'] = df['Time Seconds Finish'] / df['Distance KM']
    df['Race Location'] = df['Event'].apply(extract_location)
    df['Gender'] = df['M/F']

    df['Date'] = pd.to_datetime(df['Date'].apply(parse_date_range), format='%d.%m.%Y')
    df['YOB'] = pd.to_numeric(df['YOB'], errors='coerce').astype('Int64')
    df['Avg.Speed km/h'] = df['Avg.Speed km/h'].astype(float)
    df['Elevation Gain'] = pd.to_numeric(df['Elevation Gain'].replace({'Hm': '', 'm': ''}, regex=True), errors='coerce')

    df = df.fillna({'M/F': 'Unknown', 'Cat': 'Unknown'})
    df['Club'] = df['Club'].str.strip().str.replace(r'[^\w\s]', '', regex=True)
    df['Nat.'] = df['Nat.'].str.strip().str.upper()
    df['Age'] = df['Date'].dt.year - df['YOB']
    df = add_age_group(df)
    
    df = add_runner_statistics(df)
    df = add_elevation_gain_per_km(df)
    
    df = df.sort_values(by=['Date', 'Race Location', 'Event', 'Rank'])

    return df

In [28]:
result_df = pd.read_csv('../output/all_events_data_2022.csv', low_memory=False)

In [ ]:
result_df.head()

In [ ]:
columns_to_keep = ['Runner ID','First Name','Surname','Nat.','Gender','Age','Age Group','Cat','YOB',
                   'Race Count','Cumulative Distance KM','Avg Winner Percentage',
                   'Event ID','Event','Event Type','Date','Race Location','Elevation Gain','Elevation Gain per KM',
                   'Total Finishers','Male Finishers','Female Finishers',
                   'Rank','Rank M/F','Cat. Rank','Finish Percentage','Winner Percentage',
                   'Distance/Time','Distance KM','Terrain',
                   'Time Seconds Finish','Distance Finish','Average Speed','Avg.Speed km/h']
df_clean = df_clean[columns_to_keep]

In [ ]:
df_clean.to_csv('../tmp_clean.csv',index=False)

In [ ]:
result_df.head()

In [ ]:
df_clean.sort_values(by=['Average Speed'])

### Exclude erroneous entries
- where 0 time is recorded
- where speed is greater than x
- 

In [ ]:
df_clean[df_clean['Avg.Speed km/h']>15].sort_values(by=['Average Speed'])

In [ ]:
df[df['Performance']=="0:00:00 h"]

In [ ]:
df_clean.sort_values(by='Distance Finish', ascending=False)

### Save cleaned files

In [ ]:
results_df = load_and_concat_csv('../output/', chunksize=100000)

In [ ]:
input_folder = "../output/tmp"
output_dir = "../output/cleaned/"

all_files = glob.glob(os.path.join(input_folder, "all_events_data_*.csv"))

for file_path in all_files:
    year = re.search(r'all_events_data_(\d{4})\.csv', os.path.basename(file_path))
    if year:
        year = year.group(1)
        print(f"Processing data for year {year}")
        
        result_df = pd.read_csv(file_path, low_memory=False, encoding='utf-8')
        df_clean = clean_data(result_df)

        columns_to_keep = ['Runner ID','First Name','Surname','Nat.','Gender','Age','Age Group','Cat','YOB',
                           'Race Count','Cumulative Distance KM',
                           'Event ID','Event','Event Type','Date','Race Location','Elevation Gain','Elevation Gain per KM',
                           'Total Finishers','Male Finishers','Female Finishers',
                           'Rank','Rank M/F','Cat. Rank','Finish Percentage','Winner Percentage',
                           'Distance/Time','Distance KM','Terrain',
                           'Time Seconds Finish','Distance Finish','Average Speed','Avg.Speed km/h']
        df_clean = df_clean[columns_to_keep]

        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        output_file = os.path.join(output_dir, f"processed_data_{year}.csv")
        df_clean.to_csv(output_file, index=False)
        print(f"Processed data for year {year} saved to {output_file}")
    else:
        print(f"Skipping file {file_path} as it doesn't match the expected naming pattern.")

#### error checking

In [61]:
# Apply the cleaning function to your DataFrame
df_clean = result_df
df_clean = split_runner_name(df_clean)
df_clean = split_distance_column(df_clean)

In [62]:
df_clean.head()

,Rank,Performance,"Surname, first name",Club,Nat.,YOB,M/F,Rank M/F,Cat,Cat. Rank,...,Elevation Gain,Event ID,"Original name\nSurname, first name",source_file,hours,Surname,First Name,Distance/Time,Terrain,Event Type
0,1,8:22:27 h,"Trason, Ann",NaN,USA,1960.0,F,1.0,W40,1.0,...,NaN,14091.0,NaN,all_events_data_2000.csv,NaN,Trason,Ann,100km,track,Distance
1,1,6:25:16 h,"Trason, Ann",NaN,USA,1960.0,F,1.0,W40,1.0,...,NaN,14092.0,NaN,all_events_data_2000.csv,NaN,Trason,Ann,50mi,track,Distance
2,1,3:47:13 h,"Trason, Ann",NaN,USA,1960.0,F,1.0,W40,1.0,...,NaN,14093.0,NaN,all_events_data_2000.csv,NaN,Trason,Ann,50km,track,Distance
3,1,217.200 km,"Bakwin, Peter",NaN,USA,1962.0,M,1.0,M35,1.0,...,NaN,5356.0,NaN,all_events_data_2000.csv,NaN,Bakwin,Peter,24h,track,Time
4,2,191.600 km,"Runyan, Janet",NaN,USA,1962.0,F,1.0,W35,1.0,...,NaN,5356.0,NaN,all_events_data_2000.csv,NaN,Runyan,Janet,24h,track,Time


In [58]:
df_clean['Distance/Time'] = df_clean['Distance/Time'].astype('str')
dist = df_clean['Distance/Time'].unique()
dist.sort()

print(dist)

['07:35h  elimination race' '08:50h  elimination race'
 '100.0km  trail race' ... '9h  track' '9h  trail race' 'nan']


In [41]:
with open('Output.txt', 'w') as f:
        for value in dist:
            f.write(f"{value}\n")

In [63]:
import re

def remove_numbers_and_get_unique(unique_values):
    # Function to remove numbers from a string
    def remove_numbers(string):
        return re.sub(r'\d+(\.\d+)?', '', string).strip()

    # Remove numbers from each value and filter out empty strings
    no_numbers = [remove_numbers(str(value)) for value in unique_values if remove_numbers(str(value))]

    # Remove duplicates and sort
    unique_no_numbers = sorted(set(no_numbers))

    return unique_no_numbers

# Usage
unique_values = df_clean['Distance/Time'].astype(str).unique()
unique_without_numbers = remove_numbers_and_get_unique(unique_values)

# Print the result
print("Unique values without numbers:")
for value in unique_without_numbers:
    print(value)


Unique values without numbers:
None
d
h
km
mi


In [64]:
none_rows = df_clean[df_clean['Distance/Time'].astype(str).isin(['None', 'nan'])]

In [65]:
print(f"Number of rows with 'None' in 'Distance/Time': {len(none_rows)}")

Number of rows with 'None' in 'Distance/Time': 7


In [67]:
output_file = 'rows_with_none_distance_time.csv'
none_rows.to_csv(output_file, index=False)

In [66]:
none_rows.head()

,Rank,Performance,"Surname, first name",Club,Nat.,YOB,M/F,Rank M/F,Cat,Cat. Rank,...,Elevation Gain,Event ID,"Original name\nSurname, first name",source_file,hours,Surname,First Name,Distance/Time,Terrain,Event Type
3803929,1,5:53:08 h,"Mravlje, Dusan",NaN,SLO,1953.0,M,1.0,M35,1.0,...,NaN,66842.0,NaN,all_events_data_1990.csv,NaN,Mravlje,Dusan,None,None,None
3803930,2,6:03:59 h,"Benassi, Moreno",NaN,ITA,1949.0,M,2.0,M40,1.0,...,NaN,66842.0,NaN,all_events_data_1990.csv,NaN,Benassi,Moreno,None,None,None
3803931,3,6:04:57 h,"Ferraresi, Eliseo",NaN,ITA,1953.0,M,3.0,M35,2.0,...,NaN,66842.0,NaN,all_events_data_1990.csv,NaN,Ferraresi,Eliseo,None,None,None
3803932,4,6:09:09 h,"Pistilli, N.N.",NaN,ITA,NaN,M,4.0,NaN,1.0,...,NaN,66842.0,NaN,all_events_data_1990.csv,NaN,Pistilli,N.N.,None,None,None
3803933,5,6:09:41 h,"Lango, Viljem",NaN,SLO,1953.0,M,5.0,M35,3.0,...,NaN,66842.0,NaN,all_events_data_1990.csv,NaN,Lango,Viljem,None,None,None
